# QoS, to make Networks Dependable
<sup> <b>Inputs:</b> Vineet Upendra (vupendra@cisco.com) <b>Developed by:</b> Sarah Samuel (sasamuel@cisco.com) </sup>

## What is Quality of Service?
Quality of Service (QoS) is the technique of prioritizing traffic flows and providing preferential forwarding for higher-priority packets. The fundamental reason to implement QoS in your network is to provide guaranteed end-to-end traffic forwarding.

Modular QoS provides features that ensure better and more predictable network service by:
* supporting bandwidth allocation
* improving loss characteristics
* avoiding and managing network congestion
* metering network traffic
* setting traffic flow priorities across the network.

This notebook shows you how you can configure QoS on the network with various examples.

NETWORK TOPOLOGY:

The following topology diagram depicts the topology that is brought up in this notebook.

![Topology](./images/QoS.png)


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the simulated devices that are connected as shown in the topology diagram. The notebook refreshes the output of each cell-execution just beneath it.

The devices in the topology are brought up when you reserved the sandbox. Now you can play the code-cells to [access device consoles](#Access-Device-Consoles) and continue playing through the following steps to [configure the base network](#Configure-Base-Network), [start network traffic](#Start-Network-Traffic) and configure QoS on the above topology.

## QoS Configurations
After the simulated topology is up and running with the base configurations, we apply the QoS configurations in the below sections:
* [Create a class-map](#step5)
* [Create a policy-map](#step6)
* [Attach the traffic policy to an interface](#step7)
* [Verification of the traffic policy](#step8)
* [Example 1: IP Precedence Marking Configuration](#step9)
* [Example 2: IP DSCP Marking Configuration](#step21)
* [Example 3: QoS Group Marking Configuration](#step10)
* [Example 4: MPLS Experimental Bit Imposition Marking Configuration](#step12)
* [Example 5: MPLS Experimental Topmost Marking Configuration](#step13)

### Access Device Consoles
> Play the following cell to access the SSH console of each device in the topology.

In [ ]:
from lib.xr import *
nodes = {
         'PE1':'', 
         'P1':'',
         'P2':'', 
         'PE2':'', 
         'trex':''
        }

tb = access_device_consoles("lib/tb.yaml", nodes)

### Configure Base Network
>A simple 4 router OSPF MPLS network is the base network. Play the below cell to configure the routers. We have defined the base configuration for each of the routers in the file [xr.py](./lib/xr.py) in the respective configuration strings.

In [ ]:
out = nodes['PE1'].configure(pe1_config_str)
out = nodes['P1'].configure(p1_config_str)
out = nodes['P2'].configure(p2_config_str)
out = nodes['PE2'].configure(pe2_config_str)

### Start Network Traffic

> TREX is a software traffic generator that runs on Linux. To simulate traffic flow across the 4 router network, TREX software traffic generator ports is connected to routers PE1 and PE2

> A high priority traffic stream is injected into PE1 from TREX with the following details:

>Source IP address: 10.0.0.1

>Destination IP address: 10.1.1.1


In [ ]:
# Import the function that generates high priority traffic
from traffic.TrafficGenerator import generate_hipriority_traffic
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)
client1, client2, interact1, interact2 = generate_hipriority_traffic (trex_ipaddress, trex_port)

## <a name="step5"></a>Create a class map

>First, create the class-map by executing the below steps.

In [ ]:
# The below steps configures the class-map.
out = nodes['PE1'].configure('''
class-map match-any class1
 match dscp ef
 end-class-map
''')

>Verify the class-map with show commands.

In [ ]:
out = nodes['PE1'].execute('show class-map class1')

## <a name="step6"></a>Create a policy-map

In [ ]:
# This step configures the policy-map using the class-map created in the previous step.
out = nodes['PE1'].configure('''
policy-map policy1
 class class1
  set qos-group 4
 class class-default 
 end-policy-map
''')

> Verify the policy-map with show commands.

In [ ]:
out = nodes['PE1'].execute('show policy-map pmap-name policy1')

## <a name="step7"></a>Apply the traffic policy to an interface

In [ ]:
# Applying traffic policy to an interface in ingress direction.
# Note that policy can be applied on the interface in the ingress and egress direction.
out = nodes['PE1'].configure('''
interface FourHundredGigE0/0/0/0
 service-policy input policy1
''')

## <a name="step8"></a>Verification of the traffic policy

In [ ]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')

In [ ]:
# Check the traffic on the port
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')

## <a name="step9"></a>Example 1: IP Precedence Marking Configuration

> In the previous sections, you have already configured an input service-policy on the input interface **FourHundredGigE0/0/0/0** to set the qos-group value to 4 for all incoming traffic packets which have the **DSCP** bits set to **EF** (Expedited Forwarding). 

> Play the next cell to configure an output service policy named **out-policy1** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP precedence bit to 1 for the matched packets. 

In [ ]:
out = nodes['PE1'].configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map out-policy1
 class out-class1
  set precedence 1

interface FourHundredGigE0/0/0/1
 service-policy output out-policy1
''')

> The egress packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **out-policy1**. You can view the policy-map details and counters by playing the next cell. Note that the policy-map counter updates are slow. To see the policy-map counters incrementing, replay this cell a few times

In [ ]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')


In [ ]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')

## <a name="step21"></a>Example 2: IP DSCP Marking Configuration

> Play the next cell to configure an output service policy named **policy-dscp** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP DSCP value to 5 for the matched packets. 

In [ ]:
out = nodes['PE1'].configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map policy-dscp
 class out-class1
  set dscp 5
 class class-default
 end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-dscp
''')

> The egress traffic packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **policy-dscp**, because the input service-policy on the input interface ie **FourHundredGigE0/0/0/0** sets the qos-group value to 4 for the incoming packets.

In [ ]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')

In [ ]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')

## <a name="step10"></a>Example 3: QoS Group Marking Configuration

> In this example, you will unconfigure the input service-policy **policy1** and then configure another input service policy named **policy-qg** on the interface **FourHundredGigE0/0/0/0**. The qos-group value is set to 1 for all incoming traffic packets that match **class3**. 

In [ ]:
out = nodes['PE1'].configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-qg
 class class3
  set qos-group 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 no service-policy input policy1
 service-policy input policy-qg
''')

In [ ]:
# now Display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')

> Unconfigure the service-policy using the "no" form of the service-policy command instead of **rollback configuration**.

In [ ]:
# Remove the policy-map using "no" of the command
out = nodes['PE1'].configure('''
interface FourHundredGigE0/0/0/0
no service-policy input policy-qg
''')

## <a name="step12"></a>Example 4: MPLS Experimental Bit Imposition Marking Configuration

>In this example, you will configure an input service policy **policy-mex-imp**. This service policy is associated to a class-map **class3** through the use of the class command. This service-policy sets the MPLS EXP bits of the MPLS labels to 1 for the matched incoming packets.

In [ ]:
out = nodes['PE1'].configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-mex-imp
 class class3
  set mpls exp imposition 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 service-policy input policy-mex-imp
''')

In [ ]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')

In [ ]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')

## <a name="step13"></a>Example 5: MPLS Experimental Topmost Marking Configuration

> Play this cell to configure a service policy **policy-mex-top**. This service policy is associated to a class map **class-mex-top** through the use of the class command, and then the service policy is applied in the output direction on interface **FourHundredGigE0/0/0/1**. The MPLS EXP bits on the topmost label are set to 1 for all matched traffic packets.

In [ ]:
out = nodes['PE1'].configure('''
class-map match-any class-mex-top
 match mpls exp topmost 1
 end-class-map

policy-map policy-mex-top
 class class-mex-top
  set mpls exp topmost 2
  end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-mex-top
''')

In [ ]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')

In [ ]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')

### Clean-up Network Configurations

In [ ]:
stop_traffic(interact1, interact2, client1, client2)
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].configure(unconfig_str)